In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
# pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('house_prices.csv')

In [3]:
df.drop("Id", axis=1, inplace=True)

In [4]:
df.dropna(axis=1, inplace=True)

In [5]:
numeric_features = []
categorical_features = []
for i,t in df.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)
df = df[numeric_features]

C:\Users\jerem\AppData\Local\Temp\ipykernel_18300\2444433529.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i,t in df.dtypes.iteritems():


In [6]:
df

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,8450,7,5,2003,2003,706,0,150,856,...,0,61,0,0,0,0,0,2,2008,208500
1,20,9600,6,8,1976,1976,978,0,284,1262,...,298,0,0,0,0,0,0,5,2007,181500
2,60,11250,7,5,2001,2002,486,0,434,920,...,0,42,0,0,0,0,0,9,2008,223500
3,70,9550,7,5,1915,1970,216,0,540,756,...,0,35,272,0,0,0,0,2,2006,140000
4,60,14260,8,5,2000,2000,655,0,490,1145,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,6,5,1999,2000,0,0,953,953,...,0,40,0,0,0,0,0,8,2007,175000
1456,20,13175,6,6,1978,1988,790,163,589,1542,...,349,0,0,0,0,0,0,2,2010,210000
1457,70,9042,7,9,1941,2006,275,0,877,1152,...,0,60,0,0,0,0,2500,5,2010,266500
1458,20,9717,5,6,1950,1996,49,1029,0,1078,...,366,0,112,0,0,0,0,4,2010,142125


In [7]:
target_variable = "SalePrice"

X = df.drop(target_variable, axis=1)
Y = df.loc[:,target_variable]

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [9]:
rand_forest_rg = RandomForestRegressor(random_state=0, min_samples_split=15)
rand_forest_rg.fit(X_train, Y_train)
print(rand_forest_rg.score(X_train, Y_train))
print(rand_forest_rg.score(X_test, Y_test))

0.945322157767256
0.8366009574575247


In [10]:
cross_train = cross_val_score(rand_forest_rg, X_train, Y_train, cv=25)



In [11]:
cross_train.mean()

0.8414472358819682

In [12]:
sc = StandardScaler()

In [13]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
pca = PCA(n_components=3)

In [15]:
PC_train = pca.fit_transform(X_train)
PC_test = pca.transform(X_test)


In [16]:
pca.explained_variance_ratio_

array([0.19200008, 0.0982849 , 0.06323232])

In [17]:
PC_train = pd.DataFrame(PC_train)

In [18]:
PC_train

,0,1,2
0,4.328847,-0.581063,-0.611187
1,-3.230328,-0.387207,-1.062994
2,-2.374479,-1.772083,0.324299
3,3.683356,-2.856141,0.459000
4,2.951255,-1.151420,-1.290486
...,...,...,...
1163,4.067719,-0.132796,0.202886
1164,-1.703833,-1.313292,-0.752617
1165,-0.338436,2.871619,-0.005368
1166,1.337783,-0.216414,-1.975188


In [19]:
PC_train.std()

0    2.518220
1    1.801716
2    1.445149
dtype: float64

In [20]:
rand_forest_rg = RandomForestRegressor(random_state=0, min_samples_split=15)
rand_forest_rg.fit(PC_train, Y_train)
print(rand_forest_rg.score(PC_test, Y_test))

0.7894153015247481


In [21]:
cross_train = cross_val_score(rand_forest_rg, PC_train, Y_train, cv=25)

In [22]:
cross_train.mean()

0.8303104986609283

In [23]:
X_total = np.append(PC_train,PC_test,axis=0)
Y_total = np.append(Y_train,Y_test)

In [24]:
df_final = pd.DataFrame(X_total)
df_final["SalePrice"] = Y_total

In [25]:
px.scatter_3d(df_final, x=0, y=1, z=2,color="SalePrice" )